In [1]:
import scipy.sparse as sps
import numpy as np

try:
    import sparse_dot_mkl
    dot_product = sparse_dot_mkl.dot_product_mkl
except ImportError:
    def dot_product(x, y, dense=False, **kwargs):
        z = x @ y
        return z.A if dense and sps.issparse(z) else z 

In [2]:
np.random.seed(0)
X = np.random.randn(500, 5000)
X[X < 0.8] = 0
X = sps.csr_matrix(X)
print(f'X sparsity: {100 * (1 - X.count_nonzero() / np.prod(X.shape)):5.2f} %')

X sparsity: 78.80 %


In [3]:
X @ X.T

<500x500 sparse matrix of type '<class 'numpy.float64'>'
	with 250000 stored elements in Compressed Sparse Row format>

In [4]:
expected_result = (X @ X.T).toarray()
expected_result_tril = expected_result.copy()
expected_result_tril[np.tril_indices(expected_result.shape[0], k=-1)] = 0

In [5]:
mkl_result1 = sparse_dot_mkl.dot_product_mkl(X, X.T)
np.allclose(mkl_result1.toarray(), expected_result)

True

In [6]:
mkl_result2 = sparse_dot_mkl.gram_matrix_mkl(X, transpose=True)
np.allclose(mkl_result2.toarray(), expected_result_tril)

True

In [7]:
print("Scipy Matrix Multiplication Product:")
%timeit X @ X.T

print("MKL Matrix Multiplication Product:")
%timeit sparse_dot_mkl.dot_product_mkl(X, X.T)

print("MKL Gram Matrix Product:")
%timeit sparse_dot_mkl.dot_product_transpose_mkl(X, transpose=True)

Scipy Matrix Multiplication Product:
204 ms ± 8.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
MKL Matrix Multiplication Product:
52.5 ms ± 579 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
MKL Gram Matrix Product:
28.1 ms ± 213 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
